In [19]:
# autoreload
%load_ext autoreload
%autoreload 2

import sys
import os

# Add the parent directory of src to the path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
import torch
print(torch.__version__)
print(torch.cuda.device_count())
from unsloth import FastLanguageModel, unsloth_train

from transformers import TrainerCallback, TrainingArguments
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling, BitsAndBytesConfig
from datasets import load_dataset, Dataset
from peft import get_peft_model, LoraConfig, TaskType, PeftModel
import pprint
import json
from pathlib import Path
import transformers
import os
import numpy as np
import torch


# from src.db.manager import DBManager
from src.input_to_instructions.types import InstructionQ

# 시드 설정
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    transformers.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

# 시드값 설정
seed = 3407
set_seed(seed)


2.7.0+cu128
1


In [21]:
max_seq_length = 0     # Unsloth auto supports RoPE Scaling internally!
# dtype = None              # None for auto detection
load_in_4bit = False      # Use 4bit quantization to reduce memory usage. Can be False.
device = f"cuda"



if torch.cuda.get_device_capability()[0] >= 8:
    attn_implementation = "flash_attention_2"
    torch_dtype = torch.bfloat16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float16
# attn_implementation = "eager"
print(f"Using {attn_implementation} for attention computation.")
# QLora?

Using flash_attention_2 for attention computation.


## Load model

In [22]:
model_id = 'sh2orc/Llama-3.1-Korean-8B-Instruct'
# model_id = 'yanolja/EEVE-Korean-10.8B-v1.0'
# model_id = 'Bllossom/llama-3.2-Korean-Bllossom-3B'
# model_id = 'Saxo/Linkbricks-Horizon-AI-Korean-Gemma-2-sft-dpo-27B'
# model_id = 'Bllossom/llama-3-Korean-Bllossom-70B'
# model_id = 'LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct'

model_dir = f"/model/{model_id.replace('/', '-')}"

In [23]:
# model_id = 'Bllossom/llama-3-Korean-Bllossom-70B'
# from huggingface_hub import snapshot_download
# snapshot_download(repo_id=model_id, local_dir="70b")

In [24]:
# # Tokenizer initialization
# pretrained_model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     torch_dtype=torch_dtype,
#     cache_dir=f"{model_dir}/cache",
#     # attn_implementation=attn_implementation,
#     local_files_only=True,
#     device_map="cuda"
# )

# # tokenizer = AutoTokenizer.from_pretrained(
# #     model_id,
# #     cache_dir=f"{model_dir}/cache",
# #     local_files_only=True
# # )
# # if not os.path.exists(f"{model_dir}/config.json"):
# pretrained_model.save_pretrained(model_dir)

In [25]:
# Tokenizer initialization

pretrained_model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_id,
    # max_seq_length = max_seq_length,
    dtype = torch_dtype,
    load_in_4bit = False,
    load_in_8bit = False,
    # quantization_config=BitsAndBytesConfig(
    #     load_in_4bit=True,
    #     bnb_4bit_use_double_quant=True,
    #     bnb_4bit_quant_type="nf4",
    #     bnb_4bit_compute_dtype=torch_dtype
    #     # load_in_8bit=True,
    #     # llm_int8_enable_fp32_cpu_offload=False if not "27B" in model_id else True,
    # ),
    # device_map=device,
    cache_dir=f"{model_dir}/cache",
    attn_implementation=attn_implementation,
    # trust_remote_code=True
    # local_files_only=True
)
tokenizer.padding_side = "left"
# tokenizer.truncation_side = "left"
print(f"Pad Token id: {tokenizer.pad_token_id} and Pad Token: {tokenizer.pad_token}")
print(f"EOS Token id: {tokenizer.eos_token_id} and EOS Token: {tokenizer.eos_token}")

# if not os.path.exists(f"{model_dir}/config.json"):
#     pretrained_model.save_pretrained(model_dir)
#     tokenizer.save_pretrained(model_dir)

==((====))==  Unsloth 2025.6.2: Fast Llama patching. Transformers: 4.52.3.
   \\   /|    NVIDIA H100 80GB HBM3. Num GPUs = 1. Max memory: 79.189 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu128. CUDA: 9.0. CUDA Toolkit: 12.8. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

sh2orc/Llama-3.1-Korean-8B-Instruct does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.
Pad Token id: 128004 and Pad Token: <|finetune_right_pad_id|>
EOS Token id: 128009 and EOS Token: <|eot_id|>


# Load dataset

In [38]:
train_type = [
    "BASE", # 0

    "5SL", # 1
    "Finetuning", # 2

    "rawSQL+LM", # 3
    "QM+script", # 4

    "NoExp", # 5
    "ours" # 6
][-1]
print(f"train_type: {train_type}")

train_type: 0SL


In [27]:
from pathlib import Path

# BASE_DATASET_DIR = Path("../dataset/v5-250228-multimetadata")
# dataset_name = "v6-250306-optimizetoken"
dataset_name = "v7-250309-reduceinputanddatefunctioncall"
BASE_DATASET_DIR = Path(f"../finetuning/dataset/{dataset_name}")
print(f"BASE_DATASET_DIR: {BASE_DATASET_DIR}")
print(list(BASE_DATASET_DIR.iterdir()))

BASE_DATASET_DIR: ../finetuning/dataset/v7-250309-reduceinputanddatefunctioncall
[PosixPath('../finetuning/dataset/v7-250309-reduceinputanddatefunctioncall/prompt_v2.txt'), PosixPath('../finetuning/dataset/v7-250309-reduceinputanddatefunctioncall/refine.ipynb'), PosixPath('../finetuning/dataset/v7-250309-reduceinputanddatefunctioncall/scenario1'), PosixPath('../finetuning/dataset/v7-250309-reduceinputanddatefunctioncall/summary.ipynb'), PosixPath('../finetuning/dataset/v7-250309-reduceinputanddatefunctioncall/prompt.txt')]


In [28]:
import re

# current_metadata = json.load(open("metadata.json", "r"))


if train_type not in ["ours", "woall"]:
    common_prompt = open(BASE_DATASET_DIR / "prompt_v2.txt", "r").read()
    common_prompt = re.sub(fr"\n?<\|{train_type}\|>(.|\n)*?<\|{train_type}\|>", "", common_prompt)
else:
    common_prompt = open(BASE_DATASET_DIR / f"prompt.txt", "r").read()

    if train_type in ["woall"]:
        # search <|FI|>~~<|FI|> and remove between them
        common_prompt = re.sub(r"\n?<\|Ours\|>(.|\n)*?<\|Ours\|>", "", common_prompt)


# remove all <||>
common_prompt = re.sub(r"<\|.*?\|>", "", common_prompt)
print(common_prompt)

# print(common_prompt)

너는 유저의 HVAC 관련 질문에 답변하는 Agent의 계획을 설계하는 정확하고 훌룡한 인공지능이다. 
사용자의 질문(Input)을 받아 Agent의 Instructions를 출력해 주어야 한다.
Thinking에서는 Metadata를 바탕으로 Input을 low-level term으로 수정하고, 유저의 의도를 추측하며, 답변 계획을 세워야함.
Expectation에서는 유저가 기대할만한 답변을 추측해야함.

너는 type=q의 instruction으로 쿼리를, type=o로 python 기반 data operation을 계획할 수 있다.
type=q에서는 필요한 모든 data들을 temporal, spatial, modality(columns) 필터를 거쳐 조회해야한다.
시간과 관련된 필터링은 type=o가 아닌 type=q에서 수행한다.

expectation의 명시된 모든 variable이 type=o에서 계산되야 한다.

json형식으로 출력하며, eval() 함수를 사용할 수 있도록 괄호들과 따옴표들의 순서와 닫힘을 매우 신경써서 출력해야한다.


In [29]:
scenario_dirs = [d for d in BASE_DATASET_DIR.iterdir() if d.is_dir() and "scenario" in d.name and "metadata.json" in [f.name for f in d.iterdir()]]
print(scenario_dirs)

[PosixPath('../finetuning/dataset/v7-250309-reduceinputanddatefunctioncall/scenario1')]


In [30]:


def read_dataset(dir, path):
    # the file is originally json-list format
    # we want every first-level elements to be a string itself
    # for example, [{"Hi": "a'b'"}, {"Hi": "c'd'"}] -> ["""{"Hi": "a'b'"}""", """{"Hi": "c'd'"}"""]
    
    metadata = json.load(open(dir / "metadata.json", "r"))

    path = dir / path
    with open(path, "r", encoding="utf-8") as f:
        data = json.loads(f.read())
    
    result = []
    for d in data:
        if "v6" in dataset_name:
            if train_type in ["woall"]:
                del d["Response"]["생각"]
        elif "v7" in dataset_name:
            if train_type in ["woCoTExp"]:
                del d["Response"]["Thinking"]
                # expectations = d["Response"]["Expectations"]
                del d["Response"]["Expectations"]
                # d["Response"]["Expectations"] = expectations
            elif train_type in ["woCoT"]:
                del d["Response"]["Thinking"]
            elif train_type in ["woQM"]:
                # del d["Response"]???
                instructions = d["Response"]["Instructions"]
                instruction_qs = [i for i in instructions if i["type"] == "q"]
                final_instructions = []
                for raw_instruction in instruction_qs:
                    args = raw_instruction["args"]
                    if "table_name" in args:
                        del args["table_name"]
                    
                    if "idu_name" not in args["columns"]:
                        args["columns"].append("idu_name")
                    
                    instruction = InstructionQ(
                        args=args,
                        result_name=raw_instruction["result_name"]
                    )
                    query_string = DBManager.get_query_strings(metadata, instruction.args, exp_tag=train_type)
                    final_instructions.append({
                        "type": "q",
                        "query": query_string,
                        "result_name": raw_instruction["result_name"]
                    })

                    
                
                instructions_os = [i for i in instructions if i["type"] == "o"]
                final_instructions.extend(instructions_os)
                print("INPUT:", d["Input"])
                print(final_instructions)
                d["Response"]["Instructions"] = final_instructions
                # raise NotImplementedError
            elif train_type in ["woOp"]:
                instructions = d["Response"]["Instructions"]
                d["Response"]["Instructions"] = [i for i in instructions if i["type"] == "q"]

        
        tags = d["Tags"]["Style"]

        skip_tags = ["Reason", "Graph", "Unrelated", "Prediction"]

        skip = False
        for skip_tag in skip_tags:
            if skip_tag in tags:
                skip = True
                break
        
        if skip:
            continue

        
        result.append({"Metadata": metadata, "Input": d["Input"], "Response": json.dumps(d["Response"], ensure_ascii=False)})
    # result = [{"Input": d["Input"], "Response": json.dumps(d["Response"], ensure_ascii=False)} for d in data]
    # print(f"Read {len(result)} examples from {path}")
    # print(f"Type of result: {type(result)}")
    # print(f"Type of result[0]: {type(result[0])}")
    # print(f"Type of result[0]['Input']: {type(result[0]['Input'])}")
    # print(f"Type of result[0]['Response']: {type(result[0]['Response'])}")
    print(result)
    return result

dataset_trs = []
dataset_tss = []
for scenario_dir in scenario_dirs:
    dataset_trs.extend(read_dataset(scenario_dir, "onlyq_tr.json"))
    dataset_tss.extend(read_dataset(scenario_dir, "onlyq_ts.json"))
    # print("Warning!!!: Test set is not mutually exclusive with training set.")

dataset_sampling_rate = 1
dataset_trs = np.random.choice(dataset_trs, size=int(len(dataset_trs) * dataset_sampling_rate), replace=False)
dataset_trs = dataset_trs.tolist()
dataset_tr = Dataset.from_list(dataset_trs) # 서로 다른 메타데이터 합치면서
dataset_ts = Dataset.from_list(dataset_tss) # Mutually exclusive한 애들은 None 됨

max_seq_length = 0
def formatting_prompts_func(examples):
    convos = []
    # Iterate through each item in the batch (examples are structured as lists of values)
    for metadata, input, response in zip(examples['Metadata'], examples['Input'], examples['Response']):
        # global max_seq_length
        response.replace("    ", "")

        # print(metadata['current_datetime'])
        # print(metadata['idu_mapping'])

        answer = {
            "content": f"{response}",
            "role": "assistant"
        }
        if "llama" in model_id.lower():
            prompt = {
                "content": common_prompt,
                "role": "system"
            }
            user_input = {
                "content": f"Metadata:{metadata};Input:{input};",
                "role": "user"
            }
            convos.append([prompt, user_input, answer])
        elif "gemma" in model_id.lower():
            user_input = {
                "content": f"{common_prompt};{metadata};{input}",
                "role": "user"
            }
            convos.append([user_input, answer])
        else:
            prompt = {
                "content": common_prompt,
                "role": "system"
            }
            user_input = {
                "content": f"Metadata:{metadata};Input:{input};",
                "role": "user"
            }
            convos.append([prompt, user_input, answer])
        
        # print("Answer length: ", len(response))
        # convos.append([prompt, user_input, answer])
        
        # if len(response) + 50 > max_seq_length:
        #     max_seq_length = len(response) + len(metadata) + len(input) + 50
            # print(response)
    
    texts = [
        tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False)
        for convo in convos]
    
    # remove \n\nCutting Knowledge Date: BLAH BLAH \nToday Date: BLAH BLAH\n\n using regex
    # texts = [re.sub(r'(\nCutting Knowledge Date:.*?\nToday Date:.*?\n\n)', '', text) for text in texts]


    return {"text": texts}

dataset_tr = dataset_tr.map(formatting_prompts_func, batched=True)
dataset_ts = dataset_ts.map(formatting_prompts_func, batched=True)

max_seq_length = 10000
max_seq_length = max([len(tokenizer.encode(dataset_tr[i]['text'])) for i in range(len(dataset_tr))]) + 100
# max_seq_length += len(common_prompt)
print(max_seq_length)
print(dataset_tr[0])
print(len(dataset_tr), len(dataset_ts))
# print(f"seq length: {len(tokenizer.encode(dataset_tr[0]['text']))}")

[{'Metadata': {'idu_mapping': {'01_IB5': ['우리반'], '01_IB7': ['옆반'], '02_I81': ['앞반']}, 'modality_mapping': {'roomtemp': ['실내온도'], 'settemp': ['설정온도']}, 'current_datetime': '2022-09-30 12:00:00'}, 'Input': '어제 우리반과 옆반의 설정온도 차이 알려줘', 'Response': '{"Thinking": "어제 우리반과 옆반의 설정온도를 쿼리한 후 각 반의 평균 설정온도 및 그 차이를 계산하여 반환하면 됨.", "Expectations": ["어제 우리반의 설정온도({{어제_우리반_설정온도_평균}}℃)는 옆반의 설정온도({{어제_옆반_설정온도_평균}}℃)보다 {{어제_설정온도_평균의_차이}}℃ 높습니다."], "Instructions": [{"type": "q", "args": {"temporal": {"어제": "[DATE_TRUNC(\'day\', DATE \'CURRENT_DATE\' - INTERVAL \'1 day\'), DATE_TRUNC(\'day\', DATE \'CURRENT_DATE\'))"}, "spatials": {"우리반": "01_IB5", "옆반": "01_IB7"}, "modalities": {"설정온도": "settemp"}}}, {"type": "o", "script": ["v_어제_우리반_설정온도_df = data(t=\'어제\',s=\'우리반\',m=\'설정온도\')", "v_어제_옆반_설정온도_df = data(t=\'어제\',s=\'옆반\',m=\'설정온도\')", "v_어제_우리반_설정온도_평균 = v_어제_우리반_설정온도_df[\'설정온도\'].mean()", "v_어제_옆반_설정온도_평균 = v_어제_옆반_설정온도_df[\'설정온도\'].mean()", "v_어제_설정온도_평균의_차이 = v_어제_우리반_설정온도_평균 - v_어제_옆반_설정온도_평균"]}]}'},


[{'Metadata': {'idu_mapping': {'01_IB5': ['우리반'], '01_IB7': ['옆반'], '02_I81': ['앞반']}, 'modality_mapping': {'roomtemp': ['실내온도'], 'settemp': ['설정온도']}, 'current_datetime': '2022-09-30 12:00:00'}, 'Input': '이번주 우리반과 앞반의 평균 온도 알려줘', 'Response': '{"Thinking": "사용자는 이번주 우리반과 앞반의 실내온도의 평균을 알고싶어함. 이번주 우리반과 앞반의 실내온도 데이터를 쿼리한 후 평균값을 계산하면 됨.", "Expectations": ["이번주 우리반의 평균 실내온도({{이번주_우리반_실내온도_평균}}℃)와 앞반의 평균 실내온도({{이번주_앞반_실내온도_평균}}℃)의 차이는 {{이번주_실내온도_평균_차이}}℃입니다."], "Instructions": [{"type": "q", "args": {"temporal": {"이번주": "[DATE_TRUNC(\'week\', DATE \'CURRENT_DATE\'), DATE_TRUNC(\'week\', DATE \'CURRENT_DATE\' + INTERVAL \'1 week\'))"}, "spatials": {"우리반": "01_IB5", "앞반": "02_I81"}, "modalities": {"실내온도": "roomtemp"}}}, {"type": "o", "script": ["v_이번주_우리반_df = data(t=\'이번주\',s=\'우리반\',m=\'실내온도\')", "v_이번주_앞반_df = data(t=\'이번주\',s=\'앞반\',m=\'실내온도\')", "v_이번주_우리반_실내온도_평균 = v_이번주_우리반_df[\'실내온도\'].mean()", "v_이번주_앞반_실내온도_평균 = v_이번주_앞반_df[\'실내온도\'].mean()", "v_이번주_실내온도_평균_차이 = abs(v_이번주_우리반_실내온도_평

Map:   0%|          | 0/27 [00:00<?, ? examples/s]

Map:   0%|          | 0/11 [00:00<?, ? examples/s]

1090
{'Metadata': {'current_datetime': '2022-09-30 12:00:00', 'idu_mapping': {'01_IB5': ['우리반'], '01_IB7': ['옆반'], '02_I81': ['앞반']}, 'modality_mapping': {'roomtemp': ['실내온도'], 'settemp': ['설정온도']}}, 'Input': '지금 에너지 사용량 알려줘', 'Response': '{"Thinking": "사용자는 현재 우리반의 에너지 사용량을 알고싶어함. 현재 우리반의 에너지 사용량을 쿼리한 후 반환하면 됨.", "Expectations": ["현재 우리반의 에너지 사용량은 {{현재_우리반_에너지_사용량}}kWh입니다."], "Instructions": [{"type": "q", "args": {"temporal": {"현재": "LAST_RECORD"}, "spatials": {"우리반": "01_IB5"}, "modalities": {"에너지 사용량": "Unknown"}}}]}', 'text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\n너는 유저의 HVAC 관련 질문에 답변하는 Agent의 계획을 설계하는 정확하고 훌룡한 인공지능이다. \n사용자의 질문(Input)을 받아 Agent의 Instructions를 출력해 주어야 한다.\nThinking에서는 Metadata를 바탕으로 Input을 low-level term으로 수정하고, 유저의 의도를 추측하며, 답변 계획을 세워야함.\nExpectation에서는 유저가 기대할만한 답변을 추측해야함.\n\n너는 type=q의 instruction으로 쿼리를, type=o로 python 기반 data operation을 계획할 수 있다.\ntype=q에서는 필요한 모든 data들

In [31]:
lora_r = 200
lora_alpha = lora_r * 2
lora_repr = f"v7_r{lora_r}_a{lora_alpha}_{train_type}_tr{len(dataset_tr)}_0613"
print(lora_repr)

v7_r200_a400_ours_tr27_0613


In [32]:
peft_model = FastLanguageModel.get_peft_model(
    pretrained_model,
    r=lora_r,   # LoRA rank - suggested values: 8, 16, 32, 64, 128
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", 
                    "gate_proj", "up_proj", "down_proj",
                    # "embed_tokens",
                    # "lm_head"
                    ],
    lora_alpha=lora_alpha,
    lora_dropout=0.05,   # Supports any, but = 0 is optimized
    bias="none",      # Supports any, but = "none" is optimized
    use_gradient_checkpointing="unsloth",  # Ideal for long context tuning
    random_state=3407,
    use_rslora=False,   # Disable rank-sensitive LoRA for simpler tasks
    loftq_config=None,   # No LoftQ, for standard fine-tuning
    max_seq_length=max_seq_length,
)
# del pretrained_model


## Training config

## Train

In [33]:
torch.cuda.empty_cache()
print(len(dataset_tr))


27


In [34]:
# import shutil
# shutil.rmtree(f"{model_dir}/chkpts/{lora_repr}", ignore_errors=True)

In [35]:



per_device_train_batch_size, epochs = 60, 60 # 8
gradient_accumulation_steps = int(np.ceil(len(dataset_tr) / per_device_train_batch_size))
print(f"Gradient Accumulation Steps: {gradient_accumulation_steps}")

# clear all checkpoints

args = TrainingArguments(
    # num_train_epochs = 1,
    per_device_train_batch_size = per_device_train_batch_size,  # Controls the batch size per device
    per_device_eval_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps = gradient_accumulation_steps,  # Accumulates gradients to simulate a larger batch
    max_steps=gradient_accumulation_steps * epochs,
    # 리소스 제약때문에 batch size를 타협해야하는 경우가 발생 -> micro batch size를 줄이고,
 	# accumulated step을 늘려, 적절한 size로 gradient를 구해 weight update
    # https://www.youtube.com/watch?v=ptlmj9Y9iwE
    warmup_steps = gradient_accumulation_steps,
    learning_rate = 1e-4,             # Sets the learning rate for optimization
    fp16 = not torch.cuda.is_bf16_supported(),
    bf16 = torch.cuda.is_bf16_supported(),
    optim = "adamw_torch", # adamw_torch, adafactor, prodigy
    weight_decay = 0.01,              # Regularization term for preventing overfitting
    lr_scheduler_type = "cosine",  # Sets the learning rate scheduler
    seed = 3407,                        
    output_dir = f"{model_dir}/chkpts/{lora_repr}",  # Output directory for checkpoints and predictions     
    report_to = "none",              # Enables Weights & Biases (W&B) logging
    logging_steps = gradient_accumulation_steps,                # Sets frequency of logging to W&B
    logging_strategy = "steps",       # Logs metrics at each specified step
    eval_strategy="steps",  # enable evaluation during training
    eval_steps=gradient_accumulation_steps,
    eval_accumulation_steps=1, # 낮을수록 eval시 사용하는 메모리 줄어듦
    save_steps=gradient_accumulation_steps,
    save_strategy = "steps",               
    # load_best_model_at_end = True,    # Loads the best model at the end
    save_only_model = False,           # Saves entire model, not only weights
    # resume_from_checkpoint = f"{model_dir}/chkpts/{lora_repr}",  # Resumes training from a checkpoint
)

# def compute_metrics(eval_pred):
#     pred_ids = eval_pred.predictions
#     print(pred_ids, type(pred_ids), pred_ids.shape)
#     predictions = tokenizer.batch_decode(
#         pred_ids, 
#         skip_special_tokens=True
#     )
#     print(predictions)
#     return {"accuracy": 0}

# class CustomCallback(TrainerCallback):
#     def on_log(self, args, state, control, logs=None, **kwargs):
#         print(f"Step {state.global_step}: {logs}")

from trl import SFTTrainer

trainer = SFTTrainer(
    model = peft_model,
    processing_class = tokenizer,
    train_dataset = dataset_tr,
    eval_dataset = dataset_ts,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,        # Can make training 5x faster for short sequences.
    args = args,
    # compute_metrics = compute_metrics,
    # callbacks = [CustomCallback()]
)

Gradient Accumulation Steps: 1


Unsloth: Tokenizing ["text"]:   0%|          | 0/27 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"]:   0%|          | 0/11 [00:00<?, ? examples/s]

In [36]:

trainer_stats = unsloth_train(trainer)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 27 | Num Epochs = 60 | Total steps = 60
O^O/ \_/ \    Batch size per device = 60 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (60 x 1 x 1) = 60
 "-____-"     Trainable parameters = 524,288,000/8,554,549,248 (6.13% trained)


Step,Training Loss,Validation Loss
1,2.286900,2.193689
2,2.286900,1.381161
3,1.440200,0.790342
4,0.824200,0.357131
5,0.374600,0.230546
6,0.233500,0.154765
7,0.149600,0.120741
8,0.101100,0.106401
9,0.075100,0.097515
10,0.054800,0.092635


SafetensorError: Error while serializing: IoError(Os { code: 28, kind: StorageFull, message: "No space left on device" })

# Convert gguf

In [19]:
# # base model

# save_dir = f"{model_dir}/gguf"
# if not os.path.exists(save_dir):
#     os.makedirs(save_dir)
# command = (
#     f"python ../../llama.cpp/convert_hf_to_gguf.py "
#     # f"--base {model_dir} "              # Provide the base model config if needed
#     f"--outfile {save_dir}/base.gguf "              # Output file for the GGUF model
#     f"--outtype f16 "                      # Use f16 (or choose f32, bf16, q8_0, auto as needed)
#     f"--verbose "                            # Optional: increase logging output
#     f"{model_dir}"                      # Positional argument: path to the LoRA adapter files
# )

# print(command)

# os.system(command)

In [ ]:
step = 56

# lora_repr = "v7_r8_a16_ours_70B"
checkpoint_dir = f"{model_dir}/chkpts/{lora_repr}/checkpoint-{step}"
output_path = f"{model_dir}/gguf/{lora_repr}-checkpoint-{step}.gguf"
lora_output_dir = f"{model_dir}/lora_output/"

if not os.path.exists(f"{model_dir}/gguf"):
    os.makedirs(f"{model_dir}/gguf")
print(checkpoint_dir)

if not os.path.exists(lora_output_dir):
    os.makedirs(lora_output_dir)
print(checkpoint_dir)



/model/sh2orc-Llama-3.1-Korean-8B-Instruct/chkpts/v7_r256_a512_ours_tr17_0613/checkpoint-56
/model/sh2orc-Llama-3.1-Korean-8B-Instruct/chkpts/v7_r256_a512_ours_tr17_0613/checkpoint-56


In [ ]:
if True:
    # model = AutoModelForCausalLM.from_pretrained(
    #     checkpoint_dir,
    #     torch_dtype=torch_dtype,
    #     cache_dir=f"{model_dir}/cache",
    #     # attn_implementation=attn_implementation,
    #     local_files_only=True,
    #     device_map="cuda"
    # )

    # tokenizer = AutoTokenizer.from_pretrained(
    #     checkpoint_dir,
    #     cache_dir=f"{model_dir}/cache",
    #     local_files_only=True
    # )
    
    peft_model, tokenizer = FastLanguageModel.from_pretrained(
        checkpoint_dir,
        dtype = torch_dtype,
        attn_implementation=attn_implementation,
        load_in_4bit = False,
        load_in_8bit=False,
        cache_dir=f"{model_dir}/cache",
        local_files_only=True,
        device_map="cuda",
    )
    # FastLanguageModel.for_inference(model)

    tokenizer.padding_side = "left"

    # merge lora model and base pretrained model
    # model = model.merge_and_unload()


FileNotFoundError: /model/sh2orc-Llama-3.1-Korean-8B-Instruct/chkpts/v7_r256_a512_ours_tr17_0613/checkpoint-56/*.json (invalid repository id)

In [ ]:

# peft_model.save_pretrained_gguf(f"{lora_output_dir}/{lora_repr}-checkpoint-{step}", tokenizer, quantization_method = "q8_0")
peft_model.save_pretrained_merged(f"{lora_output_dir}/{lora_repr}-checkpoint-{step}", tokenizer, save_method="merged_16bit")
# tokenizer.save_pretrained(f"{lora_output_dir}/{lora_repr}-checkpoint-{step}")


In [ ]:
import os

# command = (
#     f"python ../../llama.cpp/convert_lora_to_gguf.py "
#     f"--base {model_dir} "              # Provide the base model config if needed
#     f"--outfile {output_path} "              # Output file for the GGUF model
#     f"--outtype f16 "                        # Use f16 (or choose f32, bf16, q8_0, auto as needed)
#     f"--verbose "                            # Optional: increase logging output
#     f"{checkpoint_dir}"                      # Positional argument: path to the LoRA adapter files
# )

command = (
    f"python ../../llama.cpp/convert_hf_to_gguf.py "
    # f"--base {model_dir} "              # Provide the base model config if needed
    f"--outfile {output_path} "              # Output file for the GGUF model
    f"--outtype f16 "                        # Use f16 (or choose f32, bf16, q8_0, auto as needed)
    f"--verbose "                            # Optional: increase logging output
    f"{lora_output_dir}/{lora_repr}-checkpoint-{step}"                      # Positional argument: path to the LoRA adapter files
)


print("Running command:", command)
os.system(command)

In [ ]:
# command = (
#     f"../../llama.cpp/build/bin/llama-quantize "
#     f"{output_path} "
#     f"{output_path.replace('.gguf', '-Q4_K_M.gguf')} "                      # Positional argument: path to the LoRA adapter files
#     f"Q4_K_M "                        # Use f16 (or choose f32, bf16, q8_0, auto as needed)
# )


# print("Running command:", command)
# os.system(command)

In [ ]:
import shutil
shutil.copy(
    f"{output_path}",
    f"../../src/i2i.gguf"
)



In [ ]:
# ! tar -cvf - ../../src/i2i.gguf | pigz -p 128 > src/i2i.tar.gz

command = (
    f"tar -cvf - ../../src/i2i.gguf | pigz -p 128 > ../../src/i2i.tar.gz"
)

print("Running command:", command)
os.system(command)
